In [ ]:
import os
import json

import pandas as pd
import requests

from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Select, HoverTool

output_notebook()

In [ ]:
user = os.getenv("GNIP_USER")
password = os.getenv("GNIP_PASSWORD")

# Twitter Enterprise: Counts

Please visit the documentation to learn more about the [Twiter Enterprise API](https://developer.twitter.com/en/docs/twitter-api/enterprise). See also [Rules and filters](https://developer.twitter.com/en/docs/twitter-api/enterprise/rules-and-filtering/building-a-rule).

In [ ]:
URL = "https://gnip-api.twitter.com/search/30day/accounts/theworldbank/prod/counts.json"

In [ ]:
# Parameters. https://papermill.readthedocs.io/en/latest/usage-parameterize.html

QUERY  = "#COVID19 lang:en"

Now, make the reques and check if HTTP 200 OK,

In [ ]:
params = {
    "query": QUERY,
    "bucket": "day",
}

r = requests.get(URL, params=params, auth=(user, password))

r.status_code == 200

Convert to `pandas.DataFrame`, 

In [ ]:
df = pd.DataFrame.from_dict(r.json()["results"])

df["date"] = pd.to_datetime(df["timePeriod"])

Plotting with `bokeh`, 

In [ ]:
p = figure(title=f"Twitter: Number of tweets ({QUERY})",
            x_axis_label="Date",
            y_axis_label="Number of tweets",
            x_axis_type='datetime', 
            width=800,
            height=600)

p.add_tools(HoverTool(
    tooltips=[
       ('(x, y)', "(@x{%F}, @y)"),],
    formatters={
        '@x': 'datetime'}))

source = ColumnDataSource(data=dict(x=df["date"].values, y=df["count"].values))
p.line(x='x', y='y', source=source, alpha=0.5)

#p.legend.location = "top_right"
#p.legend.click_policy="hide"

show(p)

## Export (Optional)

In [ ]:
#with open(f"counts/{KEYWORD}.json", "w") as f:
#    json.dump(r.json(), f)